# Robot example in colab

## Run the following cells only once when resetting the runtime.

In [ ]:
try:
  import condacolab
  condacolab.check()
except:
  !pip install -q condacolab
  import condacolab
  condacolab.install()

✨🍰✨ Everything looks OK!


# Run the following code to install the packages we will use

In [ ]:
try:
  # Check if packages are installed or not. If not, install them.
  import pinocchio
except:
  # Install the meschat-python and pybullet
  !conda install pinocchio meshcat-python pybullet

  # Install the NYU_Finger software environment.
  !pip install git+https://github.com/machines-in-motion/bullet_utils
  !pip install git+https://github.com/open-dynamic-robot-initiative/robot_properties_nyu_finger/

  # Install the code for writing controllers
  !pip install git+https://github.com/machines-in-motion/mim_data_utils
  !pip install git+https://github.com/machines-in-motion/dynamic_graph_head

# Import all libraries and create meshcat server

In [ ]:
import pinocchio as pin
import numpy as np
np.set_printoptions(precision=3, suppress=True)

from robot_properties_nyu_finger.config import NYUFingerConfig
from robot_properties_nyu_finger.wrapper import NYUFingerRobot
from bullet_utils.env import BulletEnv
import pybullet as p

from dynamic_graph_head import ThreadHead, SimHead, HoldPDController

# Start a single meshcat server instance to use for the remainder of this notebook.
import meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=['--ngrok_http_tunnel'])

viewer = meshcat.Visualizer(zmq_url=zmq_url)

You can open the visualizer by visiting the following URL:
http://41c6dde9120a.ngrok.io/static/


# Initialize the Simulator environment and setup the viewer for the created robot.

In [ ]:
class BulletEnvMeshcat(BulletEnv):
  """ Similar to the normal BulletEnv but updates a visualizer at each step. """

  def set_visualizer(self, viz):
    self._viz = viz

  def step(self, sleep=False):
    super().step(sleep)

    # Hack: Assume the robot we want to visualize is the first one.
    q = bullet_env.robots[0].get_state()[0]
    self._viz.display(q)

# Build the simulator.
bullet_env = BulletEnvMeshcat(p.DIRECT)

# Create a robot instance. This adds the robot to the simulator as well.
robot = NYUFingerRobot()

# Add the robot to the bullet_env to update the internal structure of the robot
# ate every simulation steps.
bullet_env.add_robot(robot)

# Setup the visualizer
viz = pin.visualize.MeshcatVisualizer(
    robot.pin_robot.model, robot.pin_robot.collision_model, robot.pin_robot.visual_model
)
viz.initViewer(viewer)
viz.loadViewerModel()
viz.display(np.array([0.0, 0.0, 0.0]))

bullet_env.set_visualizer(viz)

print('You should see the finger robot now when going to this page:', web_url)

You should see the finger robot now when going to this page: http://41c6dde9120a.ngrok.io/static/


In [ ]:
class PDController:
  def __init__(self, head, Kp_gain, Kd_gain):
    self.head = head
    self.Kp_gain = Kp_gain
    self.Kd_gain = Kd_gain

    self.joint_positions = head.get_sensor('joint_positions')
    self.joint_velocities = head.get_sensor('joint_velocities')

  def warmup(self, thread_head):
    self.desired_pos = self.joint_positions.copy()
    print('=== PDController::warmup')
    print('   self.desired_pos:     ', self.desired_pos)

  def update_desired_pos(self, new_target_pos):
    self.desired_pos = new_target_pos.copy()
    print('=== PDController::update_desired_pos')
    print('   self.desired_pos:     ', self.desired_pos)

  def run(self, thread_head):
    print('=== PDController::Run. ti:', thread_head.ti)
    print('   self.desired_pos:     ', self.desired_pos)
    print('   self.joint_positions: ', self.joint_positions)
    print('   self.joint_velocities:', self.joint_velocities)

    self.tau = (self.Kp_gain * (self.desired_pos - self.joint_positions) - 
                self.Kd_gain * self.joint_velocities)
    head.set_control('ctrl_joint_torques', self.tau)

    print('-> self.tau             :', self.tau)

In [ ]:
# Create a simulated head.
head = SimHead(robot, with_sliders=False)

# Reset initial position of the robot.
head.reset_state([0.0, 0.0, 0.0], [0, 0, 0])

# Create our controllers
ctrl = PDController(head, 3., 0.05)
safety_controller = HoldPDController(head, 3., 0.05, False)

# Initialize the controller infrastructure.
thread_head = ThreadHead(
    0.001, # dt.
    safety_controller, # Safety controllers.
    head, # Heads to read / write from.
    [],   # Utils.
    bullet_env # Environment to step.
)

In [ ]:
# Run simulation for 1000 steps = 1000 ms = 1 s.
thread_head.sim_run(1000)

In [ ]:
thread_head.switch_controllers(ctrl)
ctrl.update_desired_pos(np.array([0, 0.5, -1.0]))

=== PDController::warmup
   self.desired_pos:      [-0.022  0.     0.   ]
=== PDController::Run. ti: 1000
   self.desired_pos:      [-0.022  0.     0.   ]
   self.joint_positions:  [-0.022  0.     0.   ]
   self.joint_velocities: [-0.001  0.     0.   ]
-> self.tau             : [ 0. -0. -0.]
=== PDController::update_desired_pos
   self.desired_pos:      [ 0.   0.5 -1. ]


In [ ]:
from google.colab import files

# Run simulation for 100 steps = 100 ms.
thread_head.start_logging()
thread_head.sim_run(100)
log_path = thread_head.stop_logging()

files.download(log_path) 

print("You can inspect your log file using this tool:")
print("")
pritn("    https://machines-in-motion.github.io/mim_data_utils/")

  Not logging 'head' as field type '<class 'dynamic_graph_head.sim_head.SimHead'>' is unsupported
!!! ThreadHead: Start logging to file "2021-08-02_02-37-43.mds" for 30.00 seconds.
=== PDController::Run. ti: 1000
   self.desired_pos:      [ 0.   0.5 -1. ]
   self.joint_positions:  [-0.022  0.     0.   ]
   self.joint_velocities: [-0.001  0.     0.   ]
-> self.tau             : [ 0.067  1.5   -3.   ]
=== PDController::Run. ti: 1001
   self.desired_pos:      [ 0.   0.5 -1. ]
   self.joint_positions:  [-0.022  0.005 -0.015]
   self.joint_velocities: [ -0.001   4.901 -15.231]
-> self.tau             : [ 0.067  1.24  -2.193]
=== PDController::Run. ti: 1002
   self.desired_pos:      [ 0.   0.5 -1. ]
   self.joint_positions:  [-0.022  0.013 -0.042]
   self.joint_velocities: [ -0.018   8.57  -26.544]
-> self.tau             : [ 0.068  1.031 -1.547]
=== PDController::Run. ti: 1003
   self.desired_pos:      [ 0.   0.5 -1. ]
   self.joint_positions:  [-0.022  0.025 -0.076]
   self.joint_velocitie

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

You can inspect your log file using this tool:



NameError: ignored

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>